DataCamp Interactive Course

# PostgreSQL Summary Stats and Window Functions
[https://app.datacamp.com/learn/courses/postgresql-summary-stats-and-window-functions]

### Description
Have you ever wondered how data professionals use SQL to solve real-world business problems, like generating rankings, calculating moving averages and running totals, deduplicating data, or performing time intelligence? If you already know how to select, filter, order, join and group data with SQL, this course is your next step. By the end, you will be writing queries like a pro! You will learn how to create queries for analytics and data engineering with window functions, the SQL secret weapon! Using flights data, you will discover how simple it is to use window functions, and how flexible and efficient they are.

## Exercise 0.01: ETL & EDA

In [524]:
import pandas as pd
# pip install pandasql
import pandasql as ps

In [525]:
# Read the CSV file into a DataFrame
summer = pd.read_csv('summer.csv')
summer_medals = summer.copy()
print(summer_medals.shape)

(31165, 9)


In [526]:
print(summer_medals.head(2))

   Year    City     Sport Discipline          Athlete Country Gender  \
0  1896  Athens  Aquatics   Swimming     HAJOS Alfred     HUN    Men   
1  1896  Athens  Aquatics   Swimming  HERSCHMANN Otto     AUT    Men   

            Event   Medal  
0  100M Freestyle    Gold  
1  100M Freestyle  Silver  


In [527]:
# Define a SQL query
query = """
SELECT *
FROM summer_medals
LIMIT 2;
"""
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())

# Display the result
print(result)

   Year    City     Sport Discipline          Athlete Country Gender  \
0  1896  Athens  Aquatics   Swimming     HAJOS Alfred     HUN    Men   
1  1896  Athens  Aquatics   Swimming  HERSCHMANN Otto     AUT    Men   

            Event   Medal  
0  100M Freestyle    Gold  
1  100M Freestyle  Silver  


## Chapter 1: Introduction to window functions
In this chapter, you'll learn what window functions are, and the two basic window function subclauses, ORDER BY and PARTITION BY.

### Video 1.01: Introduction
[https://campus.datacamp.com/courses/postgresql-summary-stats-and-window-functions/introduction-to-window-functions?ex=1]

### Exercise 1.03: Numbering rows

Numbering rows
The simplest application for window functions is numbering rows. Numbering rows allows you to easily fetch the nth row. For example, it would be very difficult to get the 35th row in any given table if you didn't have a column with each row's number.

Copilot Explanation: The `ROW_NUMBER()` function is a window function that assigns a unique sequential integer to rows within a result set. The `OVER()` clause specifies the window over which the function operates. In this case, the `OVER()` clause is empty, meaning the row numbers are assigned across the entire result set without any specific partitioning or ordering.

This query is useful for adding a sequential row number to each row in the result set, which can be helpful for indexing, referencing, or simply keeping track of the row order.

In [529]:
# Number each row in the dataset.
query = """
SELECT
  *,
-- Assign numbers to each row
  ROW_NUMBER() OVER() AS Row_N
FROM summer_medals
ORDER BY Row_N ASC
LIMIT 3;
"""
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())

# Display the result
print(result)

''' The same but in Python: 
The range function in Python can take up to three arguments: 
start, stop, and step. In this case, it is provided with two arguments: 
start is set to 1, and stop is set to len(summer) + 1. 
The len function returns the number of rows in the DataFrame summer. 
By adding 1 to this length, the range function generates numbers from 
1 up to and including the number of rows in the DataFrame. 
The result is assigned to the new column Row_N.'''
# Assign row numbers to each row in the DataFrame summer using the range function
summer_103 = summer_medals.copy()
summer_103['Row_N'] = range(1, len(summer_103) + 1)
print(summer_103.head(3))


   Year    City     Sport Discipline           Athlete Country Gender  \
0  1896  Athens  Aquatics   Swimming      HAJOS Alfred     HUN    Men   
1  1896  Athens  Aquatics   Swimming   HERSCHMANN Otto     AUT    Men   
2  1896  Athens  Aquatics   Swimming  DRIVAS Dimitrios     GRE    Men   

                        Event   Medal  Row_N  
0              100M Freestyle    Gold      1  
1              100M Freestyle  Silver      2  
2  100M Freestyle For Sailors  Bronze      3  
   Year    City     Sport Discipline           Athlete Country Gender  \
0  1896  Athens  Aquatics   Swimming      HAJOS Alfred     HUN    Men   
1  1896  Athens  Aquatics   Swimming   HERSCHMANN Otto     AUT    Men   
2  1896  Athens  Aquatics   Swimming  DRIVAS Dimitrios     GRE    Men   

                        Event   Medal  Row_N  
0              100M Freestyle    Gold      1  
1              100M Freestyle  Silver      2  
2  100M Freestyle For Sailors  Bronze      3  


### Exercise 1.04: Numbering Olympic games in ASCENDING order
The Summer Olympics dataset contains the results of the games between 1896 and 2012. The first Summer Olympics were held in 1896, the second in 1900, and so on. What if you want to easily query the table to see in which year the 13th Summer Olympics were held? You'd need to number the rows for that.

In [530]:
print(summer_medals.columns)

Index(['Year', 'City', 'Sport', 'Discipline', 'Athlete', 'Country', 'Gender',
       'Event', 'Medal'],
      dtype='object')


In [531]:
summer_104 = summer_medals.copy()
summer_104.head(2) 

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,HAJOS Alfred,HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,HERSCHMANN Otto,AUT,Men,100M Freestyle,Silver


In [532]:
# Assign a number to each year in which Summer Olympic games were held.
query = """
SELECT
  Year,

  -- Assign numbers to each year
  ROW_NUMBER() OVER () AS Row_N
FROM (
  SELECT DISTINCT Year
  FROM summer_medals
  ORDER BY Year ASC
) AS Years
ORDER BY Year ASC;
"""
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result.head())

''' The same result with Pandas: '''
# Get distinct years and sort them in ascending order 
years_asc = summer_medals['Year'].drop_duplicates().sort_values(ascending=True).reset_index(drop=True)
# reset_index(drop=True) removes the old index and resets it to a new one starting from 0  

# Assign row numbers to each year in the DataFrame years using the range function 
years_df_asc = pd.DataFrame({'Year': years_asc, 'Row_N': range(1, len(years_asc) + 1)})
'''
1: This determines the starting value of the range, which is 1.
len(years_asc) + 1: This is the stopping value, which is exclusive, meaning the range will stop before reaching this value. By adding 1 to the length of years_asc, the range will include the last integer up to the length of years_asc.
The step value is implicitly 1, which is the default, indicating that the range will increment by 1 for each step.'''

print(years_df_asc.head())

   Year  Row_N
0  1896      1
1  1900      2
2  1904      3
3  1908      4
4  1912      5
   Year  Row_N
0  1896      1
1  1900      2
2  1904      3
3  1908      4
4  1912      5


### Video 1.05: ORDER BY
[https://campus.datacamp.com/courses/postgresql-summary-stats-and-window-functions/introduction-to-window-functions?ex=5]

### Exercise 1.06: Numbering Olympic games in DESCENDING order
We've already numbered the rows in the Summer Medals dataset. What if you need to reverse the row numbers so that the most recent Olympic games' rows have a lower number?

In [533]:
print(summer_medals.columns)

Index(['Year', 'City', 'Sport', 'Discipline', 'Athlete', 'Country', 'Gender',
       'Event', 'Medal'],
      dtype='object')


In [534]:
# Assign a number to each year in which Summer Olympic games 
# were held so that rows with the most recent years have lower row numbers.
query = """
SELECT
  Year,
  -- Assign the lowest numbers to the most recent years
  ROW_NUMBER() OVER (ORDER BY Year DESC) AS Row_N
FROM (
  SELECT DISTINCT Year
  FROM summer_medals
) AS Years
ORDER BY Year;
"""
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result.head())


''' The same result with Pandas: '''
# Get distinct years and sort them in descending order
years_desc = summer_medals['Year'].drop_duplicates().sort_values(ascending=True).reset_index(drop=True)

# Assign row numbers to each year in the DataFrame years using the range function
years_df_desc = pd.DataFrame({'Year': years_desc, 'Row_N': range(len(years_desc), 0, -1)}) # -1 is the step argument that decrements the row numbers by 1   
'''
len(years_desc): This determines the starting value of the range, which is the length of the years_desc list.
0: This is the stopping value, which is exclusive, meaning the range will stop before reaching 0.
-1: This is the step value, indicating that the range will DECREMENT by 1 for each step.'''
print(years_df_desc.head())

   Year  Row_N
0  1896     27
1  1900     26
2  1904     25
3  1908     24
4  1912     23
   Year  Row_N
0  1896     27
1  1900     26
2  1904     25
3  1908     24
4  1912     23


### Exercise 1.07: Numbering Olympic athletes by medals earned
Row numbering can also be used for ranking. For example, numbering rows and ordering by the count of medals each athlete earned in the OVER clause will assign 1 to the highest-earning medalist, 2 to the second highest-earning medalist, and so on.

In [535]:
summer.columns

Index(['Year', 'City', 'Sport', 'Discipline', 'Athlete', 'Country', 'Gender',
       'Event', 'Medal'],
      dtype='object')

In [536]:
# Step1: For each athlete, count the number of medals he or she has earned.
# Step2: Having wrapped the previous query in the Athlete_Medals CTE, 
# rank each athlete by the number of medals they've earned.
query = '''
WITH Athlete_Medals AS (
  SELECT
    -- Step1: Count the number of medals each athlete has earned
    Athlete,
    COUNT(*) AS Medals
  FROM summer_medals
  GROUP BY Athlete)

SELECT
  -- Step2: Number each athlete by how many medals they've earned
  Athlete,
  Medals,
  ROW_NUMBER() OVER (ORDER BY Medals DESC) AS Row_N
FROM Athlete_Medals
ORDER BY Medals DESC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result.head(6))

''' The same result with Pandas: '''
# Step 1: Count the number of medals each athlete has earned
medals_per_athlete = summer['Athlete'].value_counts().reset_index()
medals_per_athlete.columns = ['Athlete', 'Medals']

# Step 2: Rank each athlete by the number of medals they've earned
medals_per_athlete['Row_N'] = medals_per_athlete['Medals'].rank(method='first'
                                                      , ascending=False).astype(int)
'''The ranking is performed using the rank method from the pandas library, 
which is applied to the Medals column of the DataFrame.

The rank method is configured with the method='first' parameter, 
which assigns ranks in the order the values appear in the array, 
ensuring that ties are resolved by the ORDER OF APPEARANCE!
The ascending=False parameter indicates that the ranking should be in descending order, 
meaning that athletes with more medals will receive a higher rank 
(i.e., a lower numerical value).

After computing the ranks, the astype(int) method is used to convert 
the resulting ranks from floating-point numbers to integers. 
This conversion is necessary because the rank method returns ranks as floats by default, 
even when there are no ties. The final result is that the Row_N column contains 
integer ranks, making it easier to read and interpret the ranking of athletes based on 
their medal counts.

'''
# Sort by the number of medals in descending order
medals_per_athlete = medals_per_athlete.sort_values(by='Medals'
                                              , ascending=False).reset_index(drop=True)

# Display the result
print(medals_per_athlete.head(6))



               Athlete  Medals  Row_N
0       PHELPS Michael      22      1
1      LATYNINA Larisa      18      2
2    ANDRIANOV Nikolay      15      3
3  MANGIAROTTI Edoardo      13      4
4          ONO Takashi      13      5
5       SHAKHLIN Boris      13      6
               Athlete  Medals  Row_N
0       PHELPS Michael      22      1
1      LATYNINA Larisa      18      2
2    ANDRIANOV Nikolay      15      3
3          ONO Takashi      13      4
4  MANGIAROTTI Edoardo      13      5
5       SHAKHLIN Boris      13      6


### Exercise 1.08: Reigning weightlifting champions
A reigning champion is a champion who's won both the previous and current years' competitions. To determine if a champion is reigning, the previous and current years' results need to be in the same row, in two different columns. 

In [537]:
summer.columns

Index(['Year', 'City', 'Sport', 'Discipline', 'Athlete', 'Country', 'Gender',
       'Event', 'Medal'],
      dtype='object')

In [538]:
# Step1: Return each year's gold medalists in the Men's 69KG weightlifting competition.
# Step2: Having wrapped the previous query in the Weightlifting_Gold CTE, 
# get the previous year's champion for each year.
query = '''
WITH Weightlifting_Gold AS (
  SELECT
    -- Return each year's champions' countries
    Year,
    Country AS champion
  FROM summer_medals
  WHERE
    Discipline = 'Weightlifting' AND
    Event = '69KG' AND
    Gender = 'Men' AND
    Medal = 'Gold')

SELECT
  Year, Champion,
  -- Fetch the previous year's champion 
  -- LAG() function fetches the value from the previous row in the specified column (Champion)
  LAG(Champion) OVER (ORDER BY Year ASC) AS Last_Champion 
FROM Weightlifting_Gold
ORDER BY Year ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result.head(6))

''' The same result with Pandas: '''

   Year champion Last_Champion
0  2000      BUL          None
1  2004      CHN           BUL
2  2008      CHN           CHN
3  2012      CHN           CHN


' The same result with Pandas: '

### Video 1.09: PARTITION BY
[https://campus.datacamp.com/courses/postgresql-summary-stats-and-window-functions/introduction-to-window-functions?ex=9]

### Exercise 1.10: Reigning champions by gender
You've already fetched the previous year's champion for one event. However, if you have multiple events, genders, or other metrics as columns, you'll need to split your table into partitions to avoid having a champion from one event or gender appear as the previous champion of another event or gender.

In [539]:
summer_medals.columns

Index(['Year', 'City', 'Sport', 'Discipline', 'Athlete', 'Country', 'Gender',
       'Event', 'Medal'],
      dtype='object')

In [540]:
# Use LAG, partitioning by the gender column and ordering by year in ascending order.
query = '''
WITH Tennis_Gold AS (
  SELECT DISTINCT
    Gender, Year, Country
  FROM summer_medals
  WHERE
    Year >= 2000 AND
    Event = 'Javelin Throw' AND
    Medal = 'Gold')

SELECT
  Gender, Year,
  Country AS Champion,
  -- Fetch the previous year's champion by gender
  LAG(Country) OVER (PARTITION BY Gender
                         ORDER BY Year ASC) AS Last_Champion
FROM Tennis_Gold
ORDER BY Gender ASC, Year ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)

''' The same result using Python and the pandas library:'''
# Filter the DataFrame for the specified conditions
filtered_df = summer_medals[(summer_medals['Year'] >= 2000) & 
                            (summer_medals['Event'] == 'Javelin Throw') & 
                            (summer_medals['Medal'] == 'Gold')]

# Sort the DataFrame by Gender and Year
filtered_df = filtered_df.sort_values(by=['Gender', 'Year'])

# .shift(1) fetches the value from the previous row in the specified column (Country) 
filtered_df['Last_Champion'] = filtered_df.groupby('Gender')['Country'].shift(1)

# Rename columns to match the SQL query result
filtered_df = filtered_df.rename(columns={'Country': 'Champion'}).reset_index(drop=True)

print(filtered_df[['Gender', 'Year', 'Champion', 'Last_Champion']])

  Gender  Year Champion Last_Champion
0    Men  2000      CZE          None
1    Men  2004      NOR           CZE
2    Men  2008      NOR           NOR
3    Men  2012      TTO           NOR
4  Women  2000      NOR          None
5  Women  2004      CUB           NOR
6  Women  2008      CZE           CUB
7  Women  2012      CZE           CZE
  Gender  Year Champion Last_Champion
0    Men  2000      CZE           NaN
1    Men  2004      NOR           CZE
2    Men  2008      NOR           NOR
3    Men  2012      TTO           NOR
4  Women  2000      NOR           NaN
5  Women  2004      CUB           NOR
6  Women  2008      CZE           CUB
7  Women  2012      CZE           CZE


### Exercise 1.11: Reigning champions by gender and event
In the previous exercise, you partitioned by gender to ensure that data about one gender doesn't get mixed into data about the other gender. If you have multiple columns, however, partitioning by only one of them will still mix the results of the other columns.

In [541]:
# Return the previous champions of each year's events by gender and event.
query = '''
WITH Athletics_Gold AS (
  SELECT DISTINCT
    Gender, Year, Event, Country
  FROM summer_medals
  WHERE
    Year >= 2000 AND
    Discipline = 'Athletics' AND
    Event IN ('100M', '10000M') AND
    Medal = 'Gold')

SELECT
  Gender, Year, Event,
  Country AS Champion,
  -- Fetch the previous year's champion by gender and event
  LAG(Country) OVER (PARTITION BY Gender, Event
                         ORDER BY Year ASC) AS Last_Champion
FROM Athletics_Gold
ORDER BY Event ASC, Gender ASC, Year ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)

''' The same result using Python and the pandas library:'''
# Filter the DataFrame for the specified conditions
filtered_df = summer_medals[(summer_medals['Year'] >= 2000) & 
                            (summer_medals['Discipline'] == 'Athletics') & 
                            (summer_medals['Event'].isin(['100M', '10000M'])) & 
                            (summer_medals['Medal'] == 'Gold')]
''' filtered_df = summer_medals.query( # The same but using the query method of the DataFrame class to filter the DataFrame. 
    "Year >= 2000 and Discipline == 'Athletics' and Event in ['100M', '10000M'] and Medal == 'Gold'"
) '''

# Sort the DataFrame by Event, Gender, and Year
filtered_df = filtered_df.sort_values(by=['Event', 'Gender', 'Year'])

# Use the shift method to fetch the previous year's champion by gender and event
filtered_df['Last_Champion'] = filtered_df.groupby(['Gender', 'Event'])['Country'].shift(1)

# Rename columns to match the SQL query result
filtered_df = filtered_df.rename(columns={'Country': 'Champion'}).reset_index(drop=True)

print(filtered_df[['Gender', 'Year', 'Event', 'Champion', 'Last_Champion']])


   Gender  Year   Event Champion Last_Champion
0     Men  2000  10000M      ETH          None
1     Men  2004  10000M      ETH           ETH
2     Men  2008  10000M      ETH           ETH
3     Men  2012  10000M      GBR           ETH
4   Women  2000  10000M      ETH          None
5   Women  2004  10000M      CHN           ETH
6   Women  2008  10000M      ETH           CHN
7   Women  2012  10000M      ETH           ETH
8     Men  2000    100M      USA          None
9     Men  2004    100M      USA           USA
10    Men  2008    100M      JAM           USA
11    Men  2012    100M      JAM           JAM
12  Women  2004    100M      BLR          None
13  Women  2008    100M      JAM           BLR
14  Women  2012    100M      JAM           JAM
   Gender  Year   Event Champion Last_Champion
0     Men  2000  10000M      ETH           NaN
1     Men  2004  10000M      ETH           ETH
2     Men  2008  10000M      ETH           ETH
3     Men  2012  10000M      GBR           ETH
4   Women  20

## Chapter 2: Fetching, ranking, and paging
In this chapter, you'll learn three practical applications of window functions: fetching values from different parts of the table, ranking rows according to their values, and binning rows into different tables.


### Video 2.01: Fetching
[https://campus.datacamp.com/courses/postgresql-summary-stats-and-window-functions/fetching-ranking-and-paging?ex=1]

### Execise 2.02: Future gold medalists
Fetching functions allow you to get values from different parts of the table into one row. If you have time-ordered data, you can "peek into the future" with the LEAD fetching function. This is especially useful if you want to compare a current value to a future value.

In [542]:
summer_medals.columns

Index(['Year', 'City', 'Sport', 'Discipline', 'Athlete', 'Country', 'Gender',
       'Event', 'Medal'],
      dtype='object')

In [543]:
# For each year, fetch the current gold medalist and the gold medalist 
# 3 competitions ahead of the current row.
query = '''
WITH Discus_Medalists AS (
  SELECT DISTINCT
    Year,
    Athlete
  FROM summer_medals
  WHERE Medal = 'Gold'
    AND Event = 'Discus Throw'
    AND Gender = 'Women'
    AND Year >= 2000)

SELECT
  -- For each year, fetch the current and future medalists
  Year,
  Athlete,
  LEAD(Athlete, 3) OVER (ORDER BY Year ASC) AS Future_Champion
FROM Discus_Medalists
ORDER BY Year ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)

''' The same result using Python and the pandas library:'''
# Filter the DataFrame for the specified conditions
filtered_df = summer_medals[(summer_medals['Medal'] == 'Gold') & 
                            (summer_medals['Event'] == 'Discus Throw') & 
                            (summer_medals['Gender'] == 'Women') & 
                            (summer_medals['Year'] >= 2000)]

# Sort the DataFrame by Year
filtered_df = filtered_df.sort_values(by='Year').reset_index(drop=True)

# Use the shift method to fetch the future champion 3 competitions ahead
filtered_df['Future_Champion'] = filtered_df['Athlete'].shift(-3)

# Display the result
print(filtered_df[['Year', 'Athlete', 'Future_Champion']])





   Year                  Athlete  Future_Champion
0  2000           ZVEREVA Ellina  PERKOVIC Sandra
1  2004           SADOVA Natalya             None
2  2008  BROWN TRAFTON Stephanie             None
3  2012          PERKOVIC Sandra             None
   Year                  Athlete  Future_Champion
0  2000           ZVEREVA Ellina  PERKOVIC Sandra
1  2004           SADOVA Natalya             None
2  2008  BROWN TRAFTON Stephanie             None
3  2012          PERKOVIC Sandra             None


### Exercise 2.02: First athlete by name
It's often useful to get the first or last value in a dataset to compare all other values to it. With absolute fetching functions like FIRST_VALUE, you can fetch a value at an absolute position in the table, like its beginning or end.

In [544]:
# Return all athletes and the first athlete ordered by alphabetical order.
query = '''
WITH All_Male_Medalists AS (
  SELECT DISTINCT
    Athlete
  FROM summer_medals
  WHERE Medal = 'Gold'
    AND Gender = 'Men')

SELECT
  -- Fetch all athletes and the first athlete alphabetically
  Athlete,
  FIRST_VALUE(Athlete) OVER (
    ORDER BY Athlete ASC
  ) AS First_Athlete
FROM All_Male_Medalists;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)


''' The same result using Python and the pandas library:'''
# Fetch all athletes who have won
all_male_medalists = summer_medals.query("Medal == 'Gold' and Gender == 'Men'")['Athlete'].drop_duplicates().reset_index(drop=True) 

# Get the first athlete alphabetically
first_athlete = all_male_medalists.min() # min() returns the minimum value in the Series

print(first_athlete)

# Create a DataFrame with all athletes and the first athlete
result_df = pd.DataFrame({'Athlete': all_male_medalists})
result_df['First_Athlete'] = first_athlete

print(result_df)




                       Athlete First_Athlete
0                  AABYE Edgar   AABYE Edgar
1      AALTONEN Paavo Johannes   AABYE Edgar
2          AAS Thomas Valentin   AABYE Edgar
3           ABALMASAU Aliaksei   AABYE Edgar
4                    ABALO Luc   AABYE Edgar
...                        ...           ...
6240                ÖRVIG Thor   AABYE Edgar
6241          ÖSTERVOLD Henrik   AABYE Edgar
6242       ÖSTERVOLD Jan Olsen   AABYE Edgar
6243  ÖSTERVOLD Kristian Olsen   AABYE Edgar
6244       ÖSTERVOLD Ole Olsen   AABYE Edgar

[6245 rows x 2 columns]
AABYE Edgar
                       Athlete First_Athlete
0                 HAJOS Alfred   AABYE Edgar
1            MALOKINIS Ioannis   AABYE Edgar
2                 NEUMANN Paul   AABYE Edgar
3                 BURKE Thomas   AABYE Edgar
4                CURTIS Thomas   AABYE Edgar
...                        ...           ...
6240         NOROOZI Omid Haji   AABYE Edgar
6241              KIM Hyeonwoo   AABYE Edgar
6242              

### Exercise 2.03: Last country by name
Just like you can get the first row's value in a dataset, you can get the last row's value. This is often useful when you want to compare the most recent value to previous values.

In [545]:
# Step1: Return the year and the city in which each Olympic games were held.
# Step2: Fetch the last city in which the Olympic games were held.
query = '''
WITH Hosts AS (
  SELECT DISTINCT Year, City
    FROM Summer_Medals)

SELECT
  Year,
  City,
  -- Get the last city in which the Olympic games were held
  LAST_VALUE(City) OVER (
   ORDER BY Year ASC
   RANGE BETWEEN
     UNBOUNDED PRECEDING AND 
     -- The window starts at the first row of the partition and includes all rows up to the current row.
     UNBOUNDED FOLLOWING 
     -- The window ends at the last row of the partition and includes all rows from the current row to the last row.
  ) AS Last_City
FROM Hosts
ORDER BY Year ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)

''' The same result using Python and the pandas library: '''
# Get distinct years and cities where the Olympic games were held and sort them in ascending order
hosts = summer_medals[['Year', 'City']].drop_duplicates().sort_values(by='Year').reset_index(drop=True)

# Get the last city in which the Olympic games were held 
last_city = hosts['City'].iloc[-1]

# Add the last city to each row in the DataFrame hosts 
hosts['Last_City'] = last_city

print(hosts)




    Year                   City Last_City
0   1896                 Athens    London
1   1900                  Paris    London
2   1904               St Louis    London
3   1908                 London    London
4   1912              Stockholm    London
5   1920                Antwerp    London
6   1924                  Paris    London
7   1928              Amsterdam    London
8   1932            Los Angeles    London
9   1936                 Berlin    London
10  1948                 London    London
11  1952               Helsinki    London
12  1956  Melbourne / Stockholm    London
13  1960                   Rome    London
14  1964                  Tokyo    London
15  1968                 Mexico    London
16  1972                 Munich    London
17  1976               Montreal    London
18  1980                 Moscow    London
19  1984            Los Angeles    London
20  1988                  Seoul    London
21  1992              Barcelona    London
22  1996                Atlanta   

### Video 2.05 : Ranking
[https://campus.datacamp.com/courses/postgresql-summary-stats-and-window-functions/fetching-ranking-and-paging?ex=5]


### Exercise 2.06: Ranking athletes by medals earned
In chapter 1, you used ROW_NUMBER to rank athletes by awarded medals. However, ROW_NUMBER assigns different numbers to athletes with the same count of awarded medals, so it's not a useful ranking function; if two athletes earned the same number of medals, they should have the same rank.

In [546]:
'''Rank each athlete by the number of medals they've earned 
-- the higher the count, the higher the rank 
-- with identical numbers in case of identical values.'''
# Count the number of medals each athlete has earned
# Rank each athlete by the number of medals they've earned
query = '''
WITH Athlete_Medals AS (
  SELECT
    Athlete,
    COUNT(*) AS Medals
  FROM Summer_Medals
  GROUP BY Athlete)

SELECT
  Athlete,
  Medals,
  -- Rank athletes by the medals they've won
  RANK() OVER (ORDER BY Medals DESC) AS Rank_N
FROM Athlete_Medals
ORDER BY Medals DESC
LIMIT 10;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)

''' The same result using Python and the pandas library: '''
# Count the number of medals each athlete has earned
medals_per_athlete = summer_medals['Athlete'].value_counts().reset_index()
medals_per_athlete.columns = ['Athlete', 'Medals'] # Rename the columns 

# Rank each athlete by the number of medals they've earned
medals_per_athlete['Rank_N'] = medals_per_athlete['Medals'].rank(method='min', ascending=False).astype(int)

# Sort by the number of medals in descending order
medals_per_athlete = medals_per_athlete.sort_values(by='Medals', ascending=False).reset_index(drop=True)

# Display the result
print(medals_per_athlete.head(10))



               Athlete  Medals  Rank_N
0       PHELPS Michael      22       1
1      LATYNINA Larisa      18       2
2    ANDRIANOV Nikolay      15       3
3  MANGIAROTTI Edoardo      13       4
4          ONO Takashi      13       4
5       SHAKHLIN Boris      13       4
6     COUGHLIN Natalie      12       7
7       FISCHER Birgit      12       7
8           KATO Sawao      12       7
9         NEMOV Alexei      12       7
               Athlete  Medals  Rank_N
0       PHELPS Michael      22       1
1      LATYNINA Larisa      18       2
2    ANDRIANOV Nikolay      15       3
3          ONO Takashi      13       4
4  MANGIAROTTI Edoardo      13       4
5       SHAKHLIN Boris      13       4
6     COUGHLIN Natalie      12       7
7          TORRES Dara      12       7
8       THOMPSON Jenny      12       7
9         NEMOV Alexei      12       7


### Exercise 2.07: Ranking athletes from multiple countries
In the previous exercise, you used RANK to assign rankings to one group of athletes. In real-world data, however, you'll often find numerous groups within your data. Without partitioning your data, one group's values will influence the rankings of the others.

Also, while RANK skips numbers in case of identical values, the most natural way to assign rankings is not to skip numbers. If two countries are tied for second place, the country after them is considered to be third by most people.

In [547]:
'''Rank each country's athletes by the count of medals they've earned 
-- the higher the count, the higher the rank 
-- without skipping numbers in case of identical values.'''
# Count the number of medals each athlete has earned
# Rank each athlete by the number of medals they've earned
query = '''
WITH Athlete_Medals AS (
  SELECT
    Country, Athlete, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country IN ('JPN', 'KOR')
    AND Year >= 2000
  GROUP BY Country, Athlete
  HAVING COUNT(*) > 1 -- Filter out athletes with only 1 medal
  ) 

SELECT
  Country,
  -- Rank athletes in each country by the medals they've won
  Athlete,
  DENSE_RANK() OVER (PARTITION BY Country
                         ORDER BY Medals DESC) AS Rank_N
FROM Athlete_Medals
ORDER BY Country ASC, RANK_N ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)

''' The same result using Python and the pandas library: '''
# Filter the DataFrame for the specified conditions
filtered_df = Summer_Medals[(Summer_Medals['Country'].isin(['JPN', 'KOR'])) & 
                            (Summer_Medals['Year'] >= 2000)]

# size() counts the number of medals each athlete has earned in each country 
medals_per_athlete = filtered_df.groupby(['Country', 'Athlete']).size().reset_index(name='Medals') 

# Filter out athletes with only 1 medal 
medals_per_athlete = medals_per_athlete[medals_per_athlete['Medals'] > 1]

# Rank athletes in each country by the number of medals they've earned
medals_per_athlete['Rank_N'] = medals_per_athlete.groupby('Country')['Medals'].rank(method='dense', ascending=False).astype(int)

# Sort by Country and Rank_N in ascending order
medals_per_athlete = medals_per_athlete.sort_values(by=['Country', 'Rank_N']).reset_index(drop=True)

# Display the result
print(medals_per_athlete)



   Country          Athlete  Rank_N
0      JPN  KITAJIMA Kosuke       1
1      JPN   UCHIMURA Kohei       2
2      JPN   TACHIBANA Miya       3
3      JPN      TAKEDA Miho       3
4      JPN       ICHO Kaori       4
..     ...              ...     ...
69     KOR      OH Yong Ran       4
70     KOR      PARK Jinman       4
71     KOR    PARK Kyung-Mo       4
72     KOR    YOO Yong-Sung       4
73     KOR       YUN Ok-Hee       4

[74 rows x 3 columns]
   Country          Athlete  Medals  Rank_N
0      JPN  KITAJIMA Kosuke       7       1
1      JPN   UCHIMURA Kohei       5       2
2      JPN   TACHIBANA Miya       4       3
3      JPN      TAKEDA Miho       4       3
4      JPN       ICHO Kaori       3       4
..     ...              ...     ...     ...
69     KOR      OH Yong Ran       2       4
70     KOR      PARK Jinman       2       4
71     KOR    PARK Kyung-Mo       2       4
72     KOR    YOO Yong-Sung       2       4
73     KOR       YUN Ok-Hee       2       4

[74 rows x 4 col

### Video 2.09: Paging
[https://campus.datacamp.com/courses/postgresql-summary-stats-and-window-functions/fetching-ranking-and-paging?ex=9]

### Exercise 2.10: Paging events
There are exactly 666 unique events in the Summer Medals Olympics dataset. If you want to chunk them up to analyze them piece by piece, you'll need to split the events into groups of approximately equal size.

In [548]:
'''Split the distinct events into exactly 111 groups, 
ordered by event in alphabetical order.'''
# Split the distinct events into exactly 111 groups
query = '''
WITH Events AS (
  SELECT DISTINCT Event
  FROM Summer_Medals)
  
SELECT
  --- Split up the distinct events into 111 unique groups
  Event,
  NTILE(111) OVER (ORDER BY Event ASC) AS Page
FROM Events
ORDER BY Event ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)

''' The same result using Python and the pandas library: '''
# Get distinct events and sort them in alphabetical order
events = summer_medals['Event'].drop_duplicates().sort_values().reset_index(drop=True)

# Split the distinct events into 111 unique groups using the NTILE equivalent in pandas
# pd.qcut() splits the data into quantiles based on the number of groups specified 
# +1 to start the numbering from 1 instead of 0 
# metohd='first' to assign the same rank to identical values in the same order they appear in the array
events_df = pd.DataFrame({'Event': events, 'Page': pd.qcut(events.rank(method='first'), 111, labels=False) + 1}) 

print(events_df)






                                 Event  Page
0                              + 100KG     1
1                + 100KG (Heavyweight)     1
2          + 100KG (Super Heavyweight)     1
3                              + 105KG     1
4    + 108KG Total (Super Heavyweight)     1
..                                 ...   ...
661      York Round (100Y - 80Y - 60Y)   111
662          Épée Amateurs And Masters   111
663                    Épée Individual   111
664                       Épée Masters   111
665                          Épée Team   111

[666 rows x 2 columns]
                                 Event  Page
0                              + 100KG     1
1                + 100KG (Heavyweight)     1
2          + 100KG (Super Heavyweight)     1
3                              + 105KG     1
4    + 108KG Total (Super Heavyweight)     1
..                                 ...   ...
661      York Round (100Y - 80Y - 60Y)   111
662          Épée Amateurs And Masters   111
663                    Épée Ind

### Exercise 2.11: Top, middle, and bottom thirds
Splitting your data into thirds or quartiles is often useful to understand how the values in your dataset are spread. Getting summary statistics (averages, sums, standard deviations, etc.) of the top, middle, and bottom thirds can help you determine what distribution your values follow.

In [549]:
summer_medals.head(3)

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,HAJOS Alfred,HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,HERSCHMANN Otto,AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,DRIVAS Dimitrios,GRE,Men,100M Freestyle For Sailors,Bronze


In [550]:
# Sort by Third in ascending order, Medals in descending order, and Athlete in ascending order
# medals_per_athlete = medals_per_athlete.sort_values(by=['Third', 'Medals', 'Athlete'], ascending=[True, False, True]).reset_index(drop=True)

In [551]:
# Split the athletes into top, middle, and bottom thirds based on their count of medals.
# Get the average medals earned in each third.
query = '''
WITH Athlete_Medals AS (
  SELECT Athlete, COUNT(*) AS Medals
  FROM Summer_Medals
  GROUP BY Athlete
  HAVING COUNT(*) > 1),
  
  Thirds AS (
  SELECT
    Athlete,
    Medals,
    NTILE(3) OVER (ORDER BY Medals DESC) AS Third
  FROM Athlete_Medals)
  
SELECT
  -- Get the average medals earned in each third
  Third,
  AVG(Medals) AS Avg_Medals
FROM Thirds
GROUP BY Third
ORDER BY Third ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)

'''The same result using Python and the pandas library:'''
# Count the number of medals each athlete has earned
medals_per_athlete = summer_medals['Athlete'].value_counts().reset_index()
medals_per_athlete.columns = ['Athlete', 'Medals']

# Filter out athletes with only 1 medal
medals_per_athlete = medals_per_athlete[medals_per_athlete['Medals'] > 1]

# Split the athletes into top, middle, and bottom thirds based on their count of medals
medals_per_athlete['Third'] = pd.cut( # Split the athletes into thirds based on the number of medals they've won
    medals_per_athlete['Medals']
    .rank(method='first'), 3, labels=False) + 1 
# .rank(method='first') assigns ranks in the order the values appear in the array
# 3 is the number of bins to split the data into
# labels=False returns integer bin numbers instead of labels
# +1 to start the numbering from 1 instead of 0 

# Get the average medals earned in each third
avg_medals_per_third = medals_per_athlete.groupby('Third')['Medals'].mean().reset_index()
avg_medals_per_third.columns = ['Third', 'Avg_Medals']

# Ensure the third with the highest average medals is labeled as 1
avg_medals_per_third = avg_medals_per_third.sort_values(by='Avg_Medals', ascending=False).reset_index(drop=True)


print(avg_medals_per_third)

# Step 1: Calculate the number of medals each athlete has won
athlete_medals = (
    summer_medals.groupby('Athlete')
    .size()
    .reset_index(name='Medals')
)

''' ChatGPT solution: '''
# Filter athletes with more than 1 medal
athlete_medals = athlete_medals[athlete_medals['Medals'] > 1]

# Step 2: Sort athletes by medals in descending order and reset the index
athlete_medals = athlete_medals.sort_values(by='Medals', ascending=False).reset_index(drop=True)

# Step 3: Split the athletes into thirds based on the number of medals they've won 
# and calculate the average medals in each third
athlete_medals['Third'] = pd.cut( # Split the athletes into thirds based on the number of medals they've won
    athlete_medals.index, 
    bins=3, 
    labels=[1, 2, 3]
)

# Step 4: Calculate the average medals in each third
result = (
    athlete_medals.groupby('Third')['Medals']
    .mean() # Calculate the average medals in each third
    .reset_index(name='Avg_Medals') # Rename the column to Avg_Medals
    .sort_values(by='Third')
)

print(result)


   Third  Avg_Medals
0      1    3.786446
1      2    2.000000
2      3    2.000000
   Third  Avg_Medals
0      3    3.787464
1      1    2.000000
2      2    2.000000
  Third  Avg_Medals
0     1    3.786446
1     2    2.000000
2     3    2.000000


/var/folders/9l/hrrtsvvx7_v1n1nd2s2h8k4r0000gn/T/ipykernel_2928/3517091980.py:80: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## Chapter 3: Aggregate window functions and frames
In this chapter, you'll learn how to use aggregate functions you're familiar with, like `AVG()` and `SUM()`, as window functions, as well as how to define frames to change a window function's output.


### Video 3.01: Aggregate window functions
[https://campus.datacamp.com/courses/postgresql-summary-stats-and-window-functions/aggregate-window-functions-and-frames?ex=1]

### Exercise 3.02: Running totals of athlete medals
The running total (or cumulative sum) of a column helps you determine what each row's contribution is to the total sum.

In [552]:
# Return the athletes, the number of medals they earned, and the medals running total, 
# ordered by the athletes' names in alphabetical order.
query = '''
WITH Athlete_Medals AS (
  SELECT
    Athlete, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country = 'USA' AND Medal = 'Gold'
    AND Year >= 2000
  GROUP BY Athlete)

SELECT
  -- Calculate the running total of athlete medals
  Athlete,
  Medals,
  SUM(Medals) OVER (ORDER BY Athlete ASC) AS Max_Medals
FROM Athlete_Medals
ORDER BY Athlete ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result.head(9))

''' The same result using Python and the pandas library: '''
# Filter the DataFrame for the specified conditions
athlete_medals = summer_medals.query("Country == 'USA' and Medal == 'Gold' and Year >= 2000")

# Count the number of medals each athlete has earned
medals_per_athlete = athlete_medals['Athlete'].value_counts().reset_index()
medals_per_athlete.columns = ['Athlete', 'Medals']

# Calculate the running total of athlete medals in alphabetical order of athletes' names 
medals_per_athlete = medals_per_athlete.sort_values(by='Athlete').reset_index(drop=True)
# cumsum() calculates the cumulative sum of the Medals column
medals_per_athlete['Max_Medals'] = medals_per_athlete['Medals'].cumsum()  

# Display the result
print(medals_per_athlete.head(9))






               Athlete  Medals  Max_Medals
0  ABDUR-RAHIM Shareef       1           1
1      ABERNATHY Brent       1           2
2        ADRIAN Nathan       3           5
3         AHRENS Chris       1           6
4       AINSWORTH Kurt       1           7
5            ALLEN Ray       1           8
6          ALLEN Wyatt       1           9
7     AMBROSI Christie       1          10
8           AMICO Leah       1          11
               Athlete  Medals  Max_Medals
0  ABDUR-RAHIM Shareef       1           1
1      ABERNATHY Brent       1           2
2        ADRIAN Nathan       3           5
3         AHRENS Chris       1           6
4       AINSWORTH Kurt       1           7
5            ALLEN Ray       1           8
6          ALLEN Wyatt       1           9
7     AMBROSI Christie       1          10
8           AMICO Leah       1          11


### Exercise 3.03: Maximum country medals by year
Getting the maximum of a country's earned medals so far helps you determine whether a country has broken its medals record by comparing the current year's earned medals and the maximum so far.

In [553]:
# Return the year, country, medals, and the maximum medals earned so far for each country,
# ordered by year in ascending order.
query = '''
WITH Country_Medals AS (
  SELECT
    Year, Country, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country IN ('CHN', 'KOR', 'JPN')
    AND Medal = 'Gold' AND Year >= 2000
  GROUP BY Year, Country)

SELECT
  -- Return the max medals earned so far per country
  Country,
  Year,
  Medals,
  MAX(Medals) OVER (PARTITION BY Country
                        ORDER BY Year ASC) AS Max_Medals
FROM Country_Medals
ORDER BY Country ASC, Year ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)

''' The same result using Python and the pandas library: '''
# Filter the DataFrame for the specified conditions
filtered_df = Summer_Medals[(Summer_Medals['Country']
                            .isin(['CHN', 'KOR', 'JPN'])) & 
                            (Summer_Medals['Medal'] == 'Gold') & 
                            (Summer_Medals['Year'] >= 2000)]
# The same but using the query method of the DataFrame class to filter the DataFrame
# Summer_Medals.query("Country in ['CHN', 'KOR', 'JPN'] and Medal == 'Gold' and Year >= 2000")

# Count the number of medals each country has earned per year
# size() counts the number of medals each country has earned per year 
country_medals = filtered_df.groupby(['Year', 'Country']).size().reset_index(name='Medals') 

# Calculate the running maximum of medals earned so far per country
country_medals['Max_Medals'] = country_medals.groupby('Country')['Medals'].cummax()

# Sort by Country and Year in ascending order
country_medals = country_medals.sort_values(by=['Country', 'Year']).reset_index(drop=True)

# Display the result
print(country_medals)


   Country  Year  Medals  Max_Medals
0      CHN  2000      39          39
1      CHN  2004      52          52
2      CHN  2008      74          74
3      CHN  2012      56          74
4      JPN  2000       5           5
5      JPN  2004      21          21
6      JPN  2008      23          23
7      JPN  2012       7          23
8      KOR  2000      12          12
9      KOR  2004      14          14
10     KOR  2008      41          41
11     KOR  2012      18          41
    Year Country  Medals  Max_Medals
0   2000     CHN      39          39
1   2004     CHN      52          52
2   2008     CHN      74          74
3   2012     CHN      56          74
4   2000     JPN       5           5
5   2004     JPN      21          21
6   2008     JPN      23          23
7   2012     JPN       7          23
8   2000     KOR      12          12
9   2004     KOR      14          14
10  2008     KOR      41          41
11  2012     KOR      18          41


### Exercise 3.04: Minimum country medals by year
So far, you've seen MAX and SUM, aggregate functions normally used with GROUP BY, being used as window functions. You can also use the other aggregate functions, like MIN, as window functions.

In [554]:
# Return the year, medals earned, and minimum medals earned so far.

query = '''
WITH France_Medals AS (
  SELECT
    Year, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country = 'FRA'
    AND Medal = 'Gold' AND Year >= 2000
  GROUP BY Year)

SELECT
  Year,
  Medals,
  MIN(Medals) OVER (ORDER BY Year ASC) AS Min_Medals
FROM France_Medals
ORDER BY Year ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)

''' The same result using Python and the pandas library: '''
# Filter the DataFrame for the specified conditions
france_medals = Summer_Medals.query("Country == 'FRA' and Medal == 'Gold' and Year >= 2000")

# Count the number of medals each year
medals_per_year = france_medals.groupby('Year').size().reset_index(name='Medals')

# Calculate the running minimum of medals earned so far
medals_per_year['Min_Medals'] = medals_per_year['Medals'].cummin()

# Sort by Year in ascending order
medals_per_year = medals_per_year.sort_values(by='Year').reset_index(drop=True)

# Display the result
print(medals_per_year)


   Year  Medals  Min_Medals
0  2000      22          22
1  2004      21          21
2  2008      25          21
3  2012      30          21
   Year  Medals  Min_Medals
0  2000      22          22
1  2004      21          21
2  2008      25          21
3  2012      30          21


### Video 3.05: Frames
[https://campus.datacamp.com/courses/postgresql-summary-stats-and-window-functions/aggregate-window-functions-and-frames?ex=5]

### Exercise 3.07: Moving maximum of Scandinavian athletes' medals
Frames allow you to restrict the rows passed as input to your window function to a sliding window for you to define the start and finish.

Adding a frame to your window function allows you to calculate "moving" metrics, inputs of which slide from row to row. 

In [555]:
# Return the year, medals earned, and the maximum medals earned, 
# comparing only the current year and the next year.
query = '''
WITH Scandinavian_Medals AS (
  SELECT
    Year, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country IN ('DEN', 'NOR', 'FIN', 'SWE', 'ISL')
    AND Medal = 'Gold'
  GROUP BY Year)

SELECT
  -- Select each year's medals
  Year,
  Medals,
  -- Get the max of the current and next years'  medals
  MAX(Medals) OVER (ORDER BY Year ASC
                    ROWS BETWEEN CURRENT ROW
                    AND 1 FOLLOWING) AS Max_Medals
FROM Scandinavian_Medals
ORDER BY Year ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result.head(10))

''' The same result using Python and the pandas library: '''
# Filter the DataFrame for the specified conditions
scandinavian_medals = Summer_Medals.query("Country in ['DEN', 'NOR', 'FIN', 'SWE', 'ISL'] and Medal == 'Gold'")

# Count the number of medals each year
medals_per_year = scandinavian_medals.groupby('Year').size().reset_index(name='Medals')

# Calculate the maximum medals comparing the current year and the next year 
medals_per_year['Max_Medals'] = medals_per_year['Medals'].rolling(window=2, min_periods=1).max().shift(-1).fillna(medals_per_year['Medals']).astype(int)
# .rolling(window=2, min_periods=1) calculates the maximum of the current and next years' medals 
# .shift(-1) shifts the result by 1 row to compare the current year with the next year

# Display the result
print(medals_per_year.head(10))






   Year  Medals  Max_Medals
0  1896       1           1
1  1900       1          77
2  1908      77         141
3  1912     141         159
4  1920     159         159
5  1924      48          48
6  1928      24          24
7  1932      17          17
8  1936      15          54
9  1948      54          54
   Year  Medals  Max_Medals
0  1896       1           1
1  1900       1          77
2  1908      77         141
3  1912     141         159
4  1920     159         159
5  1924      48          48
6  1928      24          24
7  1932      17          17
8  1936      15          54
9  1948      54          54


### Exercise 3.08: Moving maximum of Chinese athletes' medals
Frames allow you to "peek" forwards or backward without first using the relative fetching functions, `LAG` and `LEAD`, to fetch previous rows' values into the current row.

In [556]:
# Return the athletes, medals earned, and the maximum medals earned, 
# comparing only the last two and current athletes, 
# ordering by athletes' names in alphabetical order.
query = '''
WITH Chinese_Medals AS (
  SELECT
    Athlete, COUNT(*) AS Medals
  FROM Summer_Medals
  WHERE
    Country = 'CHN' AND Medal = 'Gold'
    AND Year >= 2000
  GROUP BY Athlete)

SELECT
  -- Select the athletes and the medals they've earned
  Athlete,
  Medals,
  -- Get the max of the last two and current rows' medals 
  MAX(Medals) OVER (ORDER BY Athlete ASC
                    ROWS BETWEEN 2 PRECEDING
                    AND CURRENT ROW) AS Max_Medals
FROM Chinese_Medals
ORDER BY Athlete ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result.head(10))

''' The same result using Python and the pandas library: '''
# Filter the DataFrame for the specified conditions
chinese_medals = Summer_Medals.query("Country == 'CHN' and Medal == 'Gold' and Year >= 2000")

# Count the number of medals each athlete has earned
medals_per_athlete = chinese_medals['Athlete'].value_counts().reset_index()
medals_per_athlete.columns = ['Athlete', 'Medals']

# Calculate the maximum medals comparing the last two and current athletes
medals_per_athlete = medals_per_athlete.sort_values(by='Athlete').reset_index(drop=True)
medals_per_athlete['Max_Medals'] = medals_per_athlete['Medals'].rolling(window=3, min_periods=1).max().astype(int)
# .rolling(window=3, min_periods=1).max() calculates the maximum of the last two and current athletes' medals

# Display the result
print(medals_per_athlete.head(10))



        Athlete  Medals  Max_Medals
0     CAI Yalin       1           1
1       CAI Yun       1           1
2       CAO Lei       1           1
3      CAO Yuan       1           1
4     CHEN Ding       1           1
5     CHEN Jing       1           1
6       CHEN Qi       1           1
7   CHEN Ruolin       4           4
8  CHEN Xiaomin       1           4
9   CHEN Xiexia       1           4
        Athlete  Medals  Max_Medals
0     CAI Yalin       1           1
1       CAI Yun       1           1
2       CAO Lei       1           1
3      CAO Yuan       1           1
4     CHEN Ding       1           1
5     CHEN Jing       1           1
6       CHEN Qi       1           1
7   CHEN Ruolin       4           4
8  CHEN Xiaomin       1           4
9   CHEN Xiexia       1           4


### Video 3.09: Moving averages and totals
[https://campus.datacamp.com/courses/postgresql-summary-stats-and-window-functions/aggregate-window-functions-and-frames?ex=9]

### Exercise 3.11: Moving average of Russian medals
Using frames with aggregate window functions allow you to calculate many common metrics, including moving averages and totals. These metrics track the change in performance over time.

In [557]:
# Calculate the 3-year moving average of medals earned.
query = '''
WITH Russian_Medals AS (
    SELECT
        Year, COUNT(*) AS Medals
    FROM Summer_Medals
    WHERE
        Country = 'RUS'
        AND Medal = 'Gold'
        AND Year >= 1980
    GROUP BY Year)

SELECT
    Year, Medals,
    CAST(AVG(Medals) OVER -- with ROUND() function is working as well of course
        (ORDER BY Year ASC
         ROWS BETWEEN
         2 PRECEDING AND CURRENT ROW) AS INTEGER) AS Medals_MA
FROM Russian_Medals
ORDER BY Year ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)

''' The same result using Python and the pandas library: '''
# Filter the DataFrame for the specified conditions
russian_medals = Summer_Medals.query("Country == 'RUS' and Medal == 'Gold' and Year >= 1980")

# Count the number of medals each year
medals_per_year = (
        russian_medals
        .groupby('Year')
        .size() # counts the number of medals each year
        .reset_index(name='Medals')
)
# Calculate the 3-year moving average of medals earned
medals_per_year['Medals_MA'] = ( 
        medals_per_year['Medals']
        .rolling(window=3, min_periods=1)
        .mean()
        .astype(int)
)
# Display the result
print(medals_per_year)

   Year  Medals  Medals_MA
0  1996      36         36
1  2000      66         51
2  2004      47         49
3  2008      43         52
4  2012      47         45
   Year  Medals  Medals_MA
0  1996      36         36
1  2000      66         51
2  2004      47         49
3  2008      43         52
4  2012      47         45


### Exercise 3.12: Moving total of countries' medals
What if your data is split into multiple groups spread over one or more columns in the table? Even with a defined frame, if you can't somehow separate the groups' data, one group's values will affect the average of another group's values.

In [558]:
# Calculate the 3-year moving sum of medals earned per country.
query = '''
WITH Country_Medals AS (
  SELECT
    Year, Country, COUNT(*) AS Medals
  FROM Summer_Medals
  GROUP BY Year, Country)

SELECT
  Year, Country, Medals,
  -- Calculate each country's 3-game moving total
  SUM(Medals) OVER
    (PARTITION BY Country
     ORDER BY Year ASC
     ROWS BETWEEN
     2 PRECEDING AND CURRENT ROW) AS Medals_MA
FROM Country_Medals
ORDER BY Country ASC, Year ASC;
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result.head(10))

''' The same result using Python and the pandas library: '''
# Step 1: Group by 'Year' and 'Country' to count medals
country_medals = (
    summer_medals.groupby(['Year', 'Country'])
    .size()
    .reset_index(name='Medals')
)

# Step 2: Calculate the 3-game moving sum for each country using `rolling()`
country_medals['Medals_MA'] = (
    country_medals
    .sort_values(['Country', 'Year'])
    .groupby('Country')['Medals']
    .rolling(window=3, min_periods=1).sum() # Calculate the 3-game moving sum for each country 
    .reset_index(level=0, drop=True)
    .astype(int)
)

# Step 3: Sort by 'Country' and 'Year' for final output
country_medals = country_medals.sort_values(['Country', 'Year']).reset_index(drop=True)

# Display the result
print(country_medals.head(10))


   Year Country  Medals  Medals_MA
0  2012    None       4          4
1  2008     AFG       1          1
2  2012     AFG       1          2
3  1988     AHO       1          1
4  1984     ALG       2          2
5  1992     ALG       2          4
6  1996     ALG       3          7
7  2000     ALG       5         10
8  2008     ALG       2         10
9  2012     ALG       1          8
   Year Country  Medals  Medals_MA
0  2008     AFG       1          1
1  2012     AFG       1          2
2  1988     AHO       1          1
3  1984     ALG       2          2
4  1992     ALG       2          4
5  1996     ALG       3          7
6  2000     ALG       5         10
7  2008     ALG       2         10
8  2012     ALG       1          8
9  1908     ANZ      19         19


## Chapter 4: Beyond window functions
In this last chapter, you'll learn some techniques and functions that are useful when used together with window functions.

### Video 4.01: Pivoting
[https://campus.datacamp.com/courses/postgresql-summary-stats-and-window-functions/beyond-window-functions?ex=1]

### Exercise 4.02: A basic pivot
You have the following table of Pole Vault gold medalist countries by gender in 2008 and 2012.
| Gender | Year | Country |
|--------|------|---------|
| Men    | 2008 | AUS     |
| Men    | 2012 | FRA     |
| Women  | 2008 | RUS     |
| Women  | 2012 | USA     |

Pivot it by Year to get the following reshaped, cleaner table.
| Gender | 2008 | 2012 |
|--------|------|------|
| Men    | AUS  | FRA  |
| Women  | RUS  | USA  |

In [559]:
# Create the correct extension.
# Fill in the column names of the pivoted table.
# query = 
'''
-- Create the correct extension to enable CROSSTAB
CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
  SELECT
    Gender, Year, Country
  FROM Summer_Medals
  WHERE
    Year IN (2008, 2012)
    AND Medal = 'Gold'
    AND Event = 'Pole Vault'
  ORDER By Gender ASC, Year ASC
-- Fill in the correct column names for the pivoted table
$$) AS ct (Gender VARCHAR,
           "2008" VARCHAR,
           "2012" VARCHAR)
ORDER BY Gender ASC;
'''
''' 
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result)
'''
# Result is not displayed because the CROSSTAB function is not 
# supported by the SQLite database engine???


''' The same result using Python and the pandas library: '''
# Filter the DataFrame for the specified conditions
filtered_df = summer_medals[(summer_medals['Year'].isin([2008, 2012])) &
                            (summer_medals['Medal'] == 'Gold') &
                            (summer_medals['Event'] == 'Pole Vault')]

# Pivot the DataFrame
pivot_df = filtered_df.pivot(index='Gender', columns='Year', values='Country').reset_index()

# Rename the columns
pivot_df.columns.name = None # Remove the name of the columns
pivot_df.columns = ['Gender', '2008', '2012'] # Rename the columns

# Display the result
print(pivot_df)


  Gender 2008 2012
0    Men  AUS  FRA
1  Women  RUS  USA


### Exercise 4.03: Pivoting with ranking
You want to produce an easy scannable table of the rankings of the three most populous EU countries by how many gold medals they've earned in the 2004 through 2012 Olympic games. The table needs to be in this format:

| Country | 2004 | 2008 | 2012 |
|---------|------|------|------|
| FRA     | ...  | ...  | ...  |
| GBR     | ...  | ...  | ...  |
| GER     | ...  | ...  | ...  |

You'll need to count the gold medals each country has earned, produce the ranks of each country by medals earned, then pivot the table to this shape.

In [560]:
# Create the correct extension.
# Fill in the column names of the pivoted table.
# query = 
'''
CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
  WITH Country_Awards AS (
    SELECT
      Country,
      Year,
      COUNT(*) AS Awards
    FROM Summer_Medals
    WHERE
      Country IN ('FRA', 'GBR', 'GER')
      AND Year IN (2004, 2008, 2012)
      AND Medal = 'Gold'
    GROUP BY Country, Year)

  SELECT
    Country,
    Year,
    RANK() OVER
      (PARTITION BY Year
       ORDER BY Awards DESC) :: INTEGER AS rank
  FROM Country_Awards
  ORDER BY Country ASC, Year ASC;
-- Fill in the correct column names for the pivoted table
$$) AS ct (Country VARCHAR,
           "2004" INTEGER,
           "2008" INTEGER,
           "2012" INTEGER)

Order by Country ASC;
'''

# Result is not displayed because the CROSSTAB function is not 
# supported by the SQLite database engine???

''' The same result using Python and the pandas library: '''
# Filter the DataFrame for the specified conditions
filtered_df = summer_medals[(summer_medals['Country'].isin(['FRA', 'GBR', 'GER'])) &
              (summer_medals['Year'].isin([2004, 2008, 2012])) &
              (summer_medals['Medal'] == 'Gold')]

# Count the number of medals each country has earned per year
# .size() counts the number of medals each country has earned per year
country_awards = filtered_df.groupby(['Country', 'Year']).size().reset_index(name='Awards')
print(f'the number of medals each country has earned per year{country_awards}')

# Rank each country by the number of medals they've earned per year
country_awards['rank'] = country_awards.groupby('Year')['Awards'].rank(method='min', ascending=False).astype(int)

# Pivot the DataFrame to get the desired format
pivot_df = country_awards.pivot(index='Country', columns='Year', values='rank').reset_index()

# Rename the columns
pivot_df.columns.name = None
pivot_df.columns = ['Country', '2004', '2008', '2012']

# Display the result
print(pivot_df)



the number of medals each country has earned per year  Country  Year  Awards
0     FRA  2004      21
1     FRA  2008      25
2     FRA  2012      30
3     GBR  2004      17
4     GBR  2008      31
5     GBR  2012      48
6     GER  2004      41
7     GER  2008      42
8     GER  2012      45
  Country  2004  2008  2012
0     FRA     2     3     3
1     GBR     3     2     1
2     GER     1     1     2


### Video 4.04: ROLLUP and CUBE
[https://campus.datacamp.com/courses/postgresql-summary-stats-and-window-functions/beyond-window-functions?ex=4]

### Exercise 4.05: Country-level subtotals
You want to look at three Scandinavian countries' earned gold medals per country and gender in the year 2004. You're also interested in Country-level subtotals to get the total medals earned for each country, but Gender-level subtotals don't make much sense in this case, so disregard them.

In [561]:
# Count the gold medals awarded per country and gender.
# Generate Country-level gold award counts.
# query = '''
'''
-- Count the gold medals per country and gender
SELECT
  Country,
  Gender,
  COUNT(*) AS Gold_Awards
FROM Summer_Medals
WHERE
  Year = 2004
  AND Medal = 'Gold'
  AND Country IN ('DEN', 'NOR', 'SWE')
-- Generate Country-level subtotals
GROUP BY Country, ROLLUP(Gender)
ORDER BY Country ASC, Gender ASC;
'''
'''
# Execute the SQL query on the DataFrame
result = ps.sqldf(query, locals())
print(result) '''
# PandaSQLException: (sqlite3.OperationalError) no such function: ROLLUP

''' The same result using Python and the pandas library: '''
# Filter the DataFrame for the specified conditions
filtered_df = summer_medals[(summer_medals['Year'] == 2004) & 
                            (summer_medals['Medal'] == 'Gold') & 
                            (summer_medals['Country'].isin(['DEN', 'NOR', 'SWE']))]

# Count the gold medals per country and gender
gold_awards = filtered_df.groupby(['Country', 'Gender']).size().reset_index(name='Gold_Awards')

# Generate Country-level subtotals
country_totals = gold_awards.groupby('Country')['Gold_Awards'].sum().reset_index(name='Gold_Awards')

# Concatenate the detailed and subtotal DataFrames
result_df = pd.concat([gold_awards, country_totals], ignore_index=False).sort_values(by=['Country', 'Gender'], ascending=[True, True])

# Display the result
print(result_df)


  Country Gender  Gold_Awards
0     DEN    Men            4
1     DEN  Women           15
0     DEN    NaN           19
2     NOR    Men            3
3     NOR  Women            2
1     NOR    NaN            5
4     SWE    Men            4
5     SWE  Women            1
2     SWE    NaN            5


### Exercise 4.06: All group-level subtotals
You want to break down all medals awarded to Russia in the 2012 Olympic games per gender and medal type. Since the medals all belong to one country, Russia, it makes sense to generate all possible subtotals (Gender- and Medal-level subtotals), as well as a grand total.

Generate a breakdown of the medals awarded to Russia per country and medal type, including all group-level subtotals and a grand total.


In [ ]:
# Count the medals awarded per gender and medal type.
# Generate all possible group-level counts (per gender and medal type subtotals and the grand total).

# query = 
'''
-- Count the medals per gender and medal type
SELECT
  Gender,
  Medal,
  COUNT(*) AS Awards
FROM Summer_Medals
WHERE
  Year = 2012
  AND Country = 'RUS'
-- Get all possible group-level subtotals
GROUP BY CUBE(Gender, Medal)
ORDER BY Gender ASC, Medal ASC;
'''
# Execute the SQL query on the DataFrame
# result = ps.sqldf(query, locals())
# print(result) # PandaSQLException: (sqlite3.OperationalError) no such function: CUBE

''' The same result using Python and the pandas library: '''
# Filter the DataFrame for the specified conditions
filtered_df = summer_medals[(summer_medals['Year'] == 2012) & 
                            (summer_medals['Country'] == 'RUS')]

# Count the medals per gender and medal type
medal_counts = filtered_df.groupby(['Gender', 'Medal']).size().reset_index(name='Awards')

# Generate all possible group-level subtotals
gender_totals = medal_counts.groupby('Gender')['Awards'].sum().reset_index()
gender_totals['Medal'] = 'All'

medal_totals = medal_counts.groupby('Medal')['Awards'].sum().reset_index()
medal_totals['Gender'] = 'All'

grand_total = pd.DataFrame({'Gender': ['All'], 'Medal': ['All'], 'Awards': [medal_counts['Awards'].sum()]})

# Concatenate the detailed and subtotal DataFrames
result_df = pd.concat([medal_counts, gender_totals, medal_totals, grand_total], ignore_index=True).sort_values(by=['Gender', 'Medal'])

# Display the result
print(result_df)

   Gender   Medal  Awards
11    All     All     130
8     All  Bronze      51
9     All    Gold      47
10    All  Silver      32
6     Men     All      64
0     Men  Bronze      34
1     Men    Gold      23
2     Men  Silver       7
7   Women     All      66
3   Women  Bronze      17
4   Women    Gold      24
5   Women  Silver      25
